In [1]:
import numpy as np
import cv2
from imutils.video import VideoStream
import time 
import pickle
import socket
import sys
from scipy.interpolate import interp1d
import pygame
import torch
import copy
from torch.optim import Adam
from torch.nn.utils.convert_parameters import parameters_to_vector
from scipy.optimize import LinearConstraint, NonlinearConstraint, minimize

from essentials  import *
from utils import *
from panda import *

pygame 2.1.2 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
vs = VideoStream(src=0).start()
time.sleep(1.0)

frame = vs.read()
frame = cv2.flip(frame, 1)

minx = 50
maxx = 480-minx
miny = 50
maxy =640 -  miny
roi = (minx, miny, maxx, maxy)
clone = frame.copy()
image = clone[int(roi[1]):int(roi[1] + roi [3]), \
            int(roi[0]):int(roi[0] + roi[2])]

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# hsv = image

# upper_blue = np.array([140, 140 , 245])
# lower_blue = np.array([100, 100 , 200])
upper = np.array([160, 210 , 250])
lower = np.array([100, 150 , 190])
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#rgb =image
mask = cv2.inRange(rgb, lower, upper)
kernal = np.ones ((15, 15), "uint8")
red = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernal)    
result = cv2.bitwise_and(image, image, mask=mask) 
(contoursred, hierarchy) =cv2.findContours (red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for pic, contourred in enumerate (contoursred):
    area = cv2.contourArea (contourred) 
    if (area > 10):
        x, y, w, h = cv2.boundingRect (contourred)
        img = cv2.rectangle (hsv, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(img,"MARKER",(x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255))

if len(contoursred) > 0:
    # Find the biggest contour
    biggest_contour = max(contoursred, key=cv2.contourArea)

    # Find center of contour and draw filled circle
    moments = cv2.moments(biggest_contour)
    centre_of_contour = (int(moments['m10'] / moments['m00']), int(moments['m01'] / moments['m00']))
    cv2.circle(img, centre_of_contour, 2, (0, 0, 255), -1)
    # Save the center of contour so we draw line tracking it
    center_points1 = centre_of_contour
    r1 = center_points1[0]
    c1 = center_points1[1]
    y = r1-150
    x = 425-c1

    x /= 440*.446
    y /= 340*2
    # print(c1, r1)
    print("target_x={}, target_y={}".format(x,y))
else:
    x = 0
    y = 0
    print("FAILURE")
cv2.imshow('hsv', hsv)
cv2.imshow('image', image)
cv2.imshow('mask', red)
cv2.waitKey(0)
vs.stop()

target_x=0.6624541377904606, target_y=0.025


In [ ]:
# PORT = 8080
# print("[*] Connecting to Robot")
# conn = connect2robot(PORT)
# # generate the ideal trajectory (?)
# full_state = readState(conn)
# start_pos, R = joint2pose(full_state["q"])

# print("[*] Returning Home")
# HomeLevel = [0.110008,-0.345662,0.105179,-2.26974,-0.0892061,1.93603,0.851301]
# go2home(conn,HomeLevel)
# t_start = 0.0
# t_end = 10.0	
# xi0 = np.linspace(start_pos,[x,y,start_pos[2]] , 8)
# print(xi0)
# traj = Trajectory(xi0, t_start, t_end)
# delta = np.array([0.]*3)
# print("[*] Starting Interaction")


# start_time = time.time()
# last_time = time.time()
# dataset = []
# while True:

#     # measure state
#     curr_time = time.time() - start_time
#     full_state = readState(conn)
#     xyz, R = joint2pose(full_state["q"])

#     if time.time() - last_time > 0.1:   
#         dataset.append([curr_time] + list(delta))
#         last_time = time.time()

#     if curr_time > t_end:
#         print(xyz)
#         break
    
#     # # get joystick input
#     # press_A, press_B = joystick.input()
#     # if press_A:
#     # 	break

#     # is there a correction?
#     correction = False
#     applied_force = np.array([0., 0., 0.])
#     if np.linalg.norm(full_state["tau"]) > 5.0:
#         correction = True
#         F_wrench = full_state["O_F"]
#         if abs(F_wrench[0]) > 10.0:
#             applied_force[0] = -np.sign(F_wrench[0]) * 1.0
#         if abs(F_wrench[1]) > 10.0:
#             applied_force[1] = -np.sign(F_wrench[1]) * 1.0
#         if abs(F_wrench[2]) > 10.0:
#             applied_force[2] = -np.sign(F_wrench[2]) * 1.0


#     # if so, modify trajectory xi
#     if correction and curr_time < t_end:
#         delta += 0.001 * applied_force

#     # get desired position
#     desired_xyz = traj.get_waypoint(curr_time) + delta
#     # safety checks
#     if desired_xyz[0] < 0.3:
#         desired_xyz[0] = 0.3
#     if desired_xyz[0] > 0.75:
#         desired_xyz[0] = 0.75
#     if desired_xyz[1] < -0.4:
#         desired_xyz[1] = -0.4
#     if desired_xyz[1] > 0.4:
#         desired_xyz[1] = 0.4
#     if desired_xyz[2] < 0.1:
#         desired_xyz[2] = 0.1
#     if desired_xyz[2] > 0.6:
#         desired_xyz[2] = 0.6

#     # tell robot to follow xi
#     x_error = desired_xyz - xyz
#     #print(np.linalg.norm(x_error))
#     x_error = [x_error[0], x_error[1], x_error[2], 0, 0, 0]
#     q_error = xdot2qdot(x_error, full_state)	
#     send2robot(conn, q_error, "v", limit=0.3)

In [ ]:
def nothing(x):
    pass
import numpy as np
import cv2
from imutils.video import VideoStream
import time 
import pickle
import socket
import sys
from scipy.interpolate import interp1d
import pygame
import torch
import copy
from torch.optim import Adam
from torch.nn.utils.convert_parameters import parameters_to_vector
from scipy.optimize import LinearConstraint, NonlinearConstraint, minimize

from essentials  import *
from utils import *
from panda import *

vs = VideoStream(src=0).start()
time.sleep(2.0)

frame = vs.read()
frame = cv2.flip(frame, 1)
minx = 50
maxx = 480-minx
miny = 0
maxy =640 -  miny
roi = (minx, miny, maxx, maxy)
clone = frame.copy()
print(np.shape(clone))
image = clone[int(roi[1]):int(roi[1] + roi [3]), \
            int(roi[0]):int(roi[0] + roi[2])]       

#image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# hsv = image


# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
cv2.createTrackbar('RMin', 'image', 0, 255, nothing)
cv2.createTrackbar('GMin', 'image', 0, 255, nothing)
cv2.createTrackbar('BMin', 'image', 0, 255, nothing)
cv2.createTrackbar('RMax', 'image', 0, 255, nothing)
cv2.createTrackbar('GMax', 'image', 0, 255, nothing)
cv2.createTrackbar('BMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('RMax', 'image', 255)
cv2.setTrackbarPos('GMax', 'image', 255)
cv2.setTrackbarPos('BMax', 'image', 255)

# Initialize HSV min/max values
rMin = gMin = bMin = rMax = gMax = bMax = 0
prMin = pgMin = pbMin = prMax = pgMax = pbMax = 0


while(1):
    # Get current positions of all trackbars
    rMin = cv2.getTrackbarPos('RMin', 'image')
    gMin = cv2.getTrackbarPos('GMin', 'image')
    bMin = cv2.getTrackbarPos('BMin', 'image')
    rMax = cv2.getTrackbarPos('RMax', 'image')
    gMax = cv2.getTrackbarPos('GMax', 'image')
    bMax = cv2.getTrackbarPos('BMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([rMin, gMin, bMin])
    upper = np.array([rMax, gMax, bMax])

    # Convert to HSV format and color threshold
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #rgb =image
    mask = cv2.inRange(rgb, lower, upper)
    kernal = np.ones ((15, 15), "uint8")
    red = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernal)    
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if((prMin != rMin) | (pgMin != gMin) | (pbMin != bMin) | (prMax != rMax) | (pgMax != gMax) | (pbMax != bMax) ):
        #print("(rMin = %d , gMin = %d, bMin = %d), (rMax = %d , gMax = %d, bMax = %d)" % (rMin , gMin , bMin, rMax, gMax , bMax))
        phMin = rMin
        psMin = gMin
        pvMin = bMin
        phMax = rMax
        psMax = gMax
        pvMax = bMax

    # Display result image
    cv2.imshow('image', red)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
break

AttributeError: 'NoneType' object has no attribute 'copy'